In [33]:
import pandas as pd
import numpy as np
from unicodedata import category

#  DATOS DE ENTRADA (RRHH - Nómina Sucia)
data_rrhh = {
    'id_empleado': ['1001', '1002', '1003', np.nan, '1005', '1006'],
    'fecha_ingreso': ['15/01/2020', '01-Feb-2021', '2019/03/10', '20/05/2022', '15/01/2020', '11-Sep-2023'],
    'salario_mensual': ['$4,500.00', '3200.50', '€ 2.800', '5000', '$4,500.00', '3,100'],
    'departamento': ['Ventas', 'IT', 'Ventas', 'IT', 'Ventas', 'RRHH'], # Repetitivo
    'bono_anual': ['Si', 'No', 'Si', 'Si', 'No', 'No'], # Binario
    'edad': [34, 28, 45, 30, 34, 22] # Números pequeños
}

df_rrhh = pd.DataFrame(data_rrhh)
print(" Laboratorio de RRHH cargado.")
print(df_rrhh.info())

 Laboratorio de RRHH cargado.
<class 'pandas.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   id_empleado      5 non-null      str  
 1   fecha_ingreso    6 non-null      str  
 2   salario_mensual  6 non-null      str  
 3   departamento     6 non-null      str  
 4   bono_anual       6 non-null      str  
 5   edad             6 non-null      int64
dtypes: int64(1), str(5)
memory usage: 584.0 bytes
None


# Solución 1: Limpieza Financiera (Strings a Floats)

In [34]:
# Convertir columna 'salario_mensual' a String para la eliminación de símbolos como $,€ y ,
df_rrhh['salario_mensual'] = (
    df_rrhh['salario_mensual']
    .astype(str)
    .str.replace(r'[€$]', '', regex=True)   # quitar símbolos
    .str.replace('.', '', regex=False)      # quitar miles (formato europeo)
    .str.replace(',', '.', regex=False)
)

#Después convertimos a decimales (float)
df_rrhh['salario_mensual'] = df_rrhh['salario_mensual'].astype(float)

print("Salarios convertidos:")
print(df_rrhh['salario_mensual'].sum())


Salarios convertidos:
327862.1


# Solución 2: El Calendario Corporativo (Fechas Mixtas)

In [35]:
# Conventir a fecha
df_rrhh['fecha_ingreso'] = pd.to_datetime(df_rrhh['fecha_ingreso'],format='mixed',dayfirst=True)
print(df_rrhh)
print("---- TIPO DE DATO-----")
print(df_rrhh['fecha_ingreso'].dtypes)

  id_empleado fecha_ingreso  salario_mensual departamento bono_anual  edad
0        1001    2020-01-15              4.5       Ventas         Si    34
1        1002    2021-02-01         320050.0           IT         No    28
2        1003    2019-10-03           2800.0       Ventas         Si    45
3         NaN    2022-05-20           5000.0           IT         Si    30
4        1005    2020-01-15              4.5       Ventas         No    34
5        1006    2023-09-11              3.1         RRHH         No    22
---- TIPO DE DATO-----
datetime64[us]


# Solución 3: Optimización Categórica (Ahorro de RAM)


In [36]:
#Convertir a Category para ahorrar memoria
df_rrhh['departamento']=df_rrhh['departamento'].astype('category')
print(df_rrhh['departamento'])

0    Ventas
1        IT
2    Ventas
3        IT
4    Ventas
5      RRHH
Name: departamento, dtype: category
Categories (3, str): ['IT', 'RRHH', 'Ventas']


# Solución 4:Banderas Booleanas (Lógica Binaria)

In [37]:
mapa_bono = {'Si':True, 'No':False}
df_rrhh['bono_anual'] = df_rrhh['bono_anual'].map(mapa_bono).astype('bool')
print(df_rrhh['bono_anual'])

0     True
1    False
2     True
3     True
4    False
5    False
Name: bono_anual, dtype: bool


# Solución 5: IDs Perdidos (Modern Pandas Int64)

In [38]:
#Convertir a Int64
df_rrhh['id_empleado'] = df_rrhh['id_empleado'].astype('Int64')
print(df_rrhh['id_empleado'])

0    1001
1    1002
2    1003
3    <NA>
4    1005
5    1006
Name: id_empleado, dtype: Int64


# Solución 6: Downcasting (Micro-Optimización)

In [39]:
df_rrhh['edad'] = df_rrhh['edad'].astype('int8')
print("----- USO DE MEMORIA(int8)----")
print((df_rrhh.info()))

----- USO DE MEMORIA(int8)----
<class 'pandas.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_empleado      5 non-null      Int64         
 1   fecha_ingreso    6 non-null      datetime64[us]
 2   salario_mensual  6 non-null      float64       
 3   departamento     6 non-null      category      
 4   bono_anual       6 non-null      bool          
 5   edad             6 non-null      int8          
dtypes: Int64(1), bool(1), category(1), datetime64[us](1), float64(1), int8(1)
memory usage: 337.0 bytes
None
